# Carla notebook

In [1]:
# This file serves as a starting template for other scripts.
import glob
import os
import sys
import random
import time
import numpy as np
import cv2
import open3d as o3d    # Open3D is used for 3D visualization of point clouds
from matplotlib import cm   # Colormap for point cloud visualization

# Add the carla egg file to the python path
try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla

client = carla.Client('localhost', 2000)
# client.reload_world()
world = client.get_world()

blueprint_library = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]
random_start_point = random.choice(spawn_points)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Create Actor List
# Function to clean up all actors
def cleanUp():
    for actor in actor_list:
        actor.destroy()
    print("All cleaned up!")

actor_list = []

In [3]:
# Spawn vehicle
vehicle_bp = blueprint_library.filter('vehicle.*')[0]
# vehicle = world.spawn_actor(vehicle_bp, random_start_point)
vehicle = world.spawn_actor(vehicle_bp, start_point)
actor_list.append(vehicle)

In [4]:
# Display available sensors
for bp in blueprint_library.filter('sensor'):
    print(bp.id)

sensor.other.collision
sensor.camera.depth
sensor.camera.optical_flow
sensor.camera.normals
sensor.other.lane_invasion
sensor.camera.dvs
sensor.other.imu
sensor.other.gnss
sensor.other.obstacle
sensor.other.radar
sensor.lidar.ray_cast_semantic
sensor.lidar.ray_cast
sensor.camera.rgb
sensor.camera.semantic_segmentation
sensor.other.rss
sensor.camera.instance_segmentation


In [4]:
# Spawn sensors
# Spawn RGB camera
camera_bp = blueprint_library.find('sensor.camera.rgb')
# Move the camera to a certain position in space before attaching it to the vehicle
camera_init = carla.Transform(carla.Location(z=1.6, x=0.4))
camera_sensor = world.spawn_actor(camera_bp, camera_init, attach_to=vehicle)
actor_list.append(camera_sensor)

# # Test camera position then clean up
# time.sleep(0.2)
# spectator.set_transform(camera_sensor.get_transform())
# camera_sensor.destroy()

# Spawn Semantic Camera
semantic_camera_bp = blueprint_library.find('sensor.camera.semantic_segmentation')
semantic_camera_sensor = world.spawn_actor(semantic_camera_bp, camera_init, attach_to=vehicle)
actor_list.append(semantic_camera_sensor)
# Spawn instance camera
instance_camera_bp = blueprint_library.find('sensor.camera.instance_segmentation')
instance_camera_sensor = world.spawn_actor(instance_camera_bp, camera_init, attach_to=vehicle)
actor_list.append(instance_camera_sensor)
# Spawn depth camera
depth_camera_bp = blueprint_library.find('sensor.camera.depth')
depth_camera_sensor = world.spawn_actor(depth_camera_bp, camera_init, attach_to=vehicle)
actor_list.append(depth_camera_sensor)
# Spawn dvs (dynamic vision sensor) camera
dvs_camera_bp = blueprint_library.find('sensor.camera.dvs')
dvs_camera_sensor = world.spawn_actor(dvs_camera_bp, camera_init, attach_to=vehicle)
actor_list.append(dvs_camera_sensor)
# Spawn optical flow camera
optic_camera_bp = blueprint_library.find('sensor.camera.optical_flow')
optic_camera_sensor = world.spawn_actor(optic_camera_bp, camera_init, attach_to=vehicle)
actor_list.append(optic_camera_sensor)


# Spawn GNSS
gnss_bp = blueprint_library.find('sensor.other.gnss')
# attach gnss sensor to the vehicle
gnss_sensor = world.spawn_actor(gnss_bp, carla.Transform(), attach_to=vehicle)
actor_list.append(gnss_sensor)

In [5]:
# Callback functions for sensors
# RGB Camera callback
def camera_callback(image, data_dict):
    # Convert the image to a numpy array and store it in the data dictionary with 4 as rgb and alpha channel
    data_dict['rgb_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Semantic Camera callback
def semantic_camera_callback(image, data_dict):
    image.convert(carla.ColorConverter.CityScapesPalette)
    data_dict['sem_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
# Instance Camera callback
def instance_camera_callback(image, data_dict):
    data_dict['inst_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# Depth Camera callback
def depth_camera_callback(image, data_dict):
    # gives forground images more contrast
    image.convert(carla.ColorConverter.LogarithmicDepth)
    data_dict['depth_image'] = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))

# DVS Camera callback
def dvs_camera_callback(data, data_dict):
    dvs_events = np.frombuffer(data.raw_data, dtype=np.dtype([
                ('x', np.uint16), ('y', np.uint16), ('t', np.int64), ('pol', np.bool_)]))
    data_dict['dvs_image'] = np.zeros((data.height, data.width, 4), dtype=np.uint8)
    dvs_img = np.zeros((data.height, data.width, 3), dtype=np.uint8)
    dvs_img[dvs_events[:]['y'], dvs_events[:]['x'], dvs_events[:]['pol'] * 2] = 255
    data_dict['dvs_image'][:, :, 0:3] = dvs_img

# Optical Flow Camera callback
def optic_camera_callback(data, data_dict):
    image = data.get_color_coded_flow()
    img = np.reshape(np.copy(image.raw_data), (image.height, image.width, 4))
    # Set the alpha channel to 255
    img[:, :, 3] = 255
    data_dict['opt_image'] = img

# GNSS callback
def gnss_callback(data):
    data_dict['gnss'] = [data.latitude, data.longitude, data.altitude]

In [8]:
# Extract data from sensors
image_width = camera_bp.get_attribute('image_size_x').as_int()
image_height = camera_bp.get_attribute('image_size_y').as_int()

sensor_data = {
    'rgb_image': np.zeros((image_height, image_width, 4)),
    'sem_image': np.zeros((image_height, image_width, 4)),
    'inst_image': np.zeros((image_height, image_width, 4)),
    'depth_image': np.zeros((image_height, image_width, 4)),
    'dvs_image': np.zeros((image_height, image_width, 4)),
    'opt_image': np.zeros((image_height, image_width, 4)),
}

cv2.namedWindow('All Cameras', cv2.WINDOW_AUTOSIZE)

# Set scaling factor for window size
scale_factor = 0.7

def resize_image(image, scale_factor):
    width = int(image.shape[1] * scale_factor)
    height = int(image.shape[0] * scale_factor)
    return cv2.resize(image, (width, height))


# Concatenate each image into a single image
top_row = np.concatenate((sensor_data['rgb_image'], sensor_data['sem_image'], sensor_data['inst_image']), axis=1)
lower_row = np.concatenate((sensor_data['depth_image'], sensor_data['dvs_image'], sensor_data['opt_image']), axis=1)
tiled = np.concatenate((top_row, lower_row), axis=0)

cv2.imshow('All Cameras', tiled)
cv2.waitKey(1)

# Set up callbacks
camera_sensor.listen(lambda image: camera_callback(image, sensor_data))
semantic_camera_sensor.listen(lambda image: semantic_camera_callback(image, sensor_data))
instance_camera_sensor.listen(lambda image: instance_camera_callback(image, sensor_data))
depth_camera_sensor.listen(lambda image: depth_camera_callback(image, sensor_data))
dvs_camera_sensor.listen(lambda image: dvs_camera_callback(image, sensor_data))
optic_camera_sensor.listen(lambda image: optic_camera_callback(image, sensor_data))

while True:

    # Resize images before concatenation
    rgb_resized = resize_image(sensor_data['rgb_image'], scale_factor)
    sem_resized = resize_image(sensor_data['sem_image'], scale_factor)
    inst_resized = resize_image(sensor_data['inst_image'], scale_factor)
    depth_resized = resize_image(sensor_data['depth_image'], scale_factor)
    dvs_resized = resize_image(sensor_data['dvs_image'], scale_factor)
    opt_resized = resize_image(sensor_data['opt_image'], scale_factor)

    # Concatenate each image into a single image
    top_row = np.concatenate((rgb_resized, sem_resized, inst_resized), axis=1)
    lower_row = np.concatenate((depth_resized, dvs_resized, opt_resized), axis=1)
    tiled = np.concatenate((top_row, lower_row), axis=0)

    cv2.imshow('All Cameras', tiled)
    if cv2.waitKey(1) == ord('q'):
        break
    
camera_sensor.stop()
semantic_camera_sensor.stop()
instance_camera_sensor.stop()
depth_camera_sensor.stop()
dvs_camera_sensor.stop()
optic_camera_sensor.stop()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 

In [11]:
# Cause vehicle to drive forward for 3 seconds
vehicle.apply_control(carla.VehicleControl(throttle=1.0, steer=0.0))
time.sleep(3)
# Stop vehicle
vehicle.apply_control(carla.VehicleControl(throttle=0.0, steer=0.0))


In [9]:
# Create OpenCV window to render the camera data
cv2.namedWindow('Camera'), cv2.WINDOW_AUTOSIZE
cv2. imshow('Camera', camera_data['image'])
cv2.waitKey(1)

while True:
    cv2.imshow('Camera', camera_data['image'])
    # Break the loop if the user presses the 'q' key with window focused
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()



In [36]:
# Get vehicle position on map
# Note that transform means positioning of the car, not changes in the car
vehicle_pos = vehicle.get_transform()
print(vehicle_pos)
# get the vehicle's gnss location



Transform(Location(x=-29.336229, y=24.564341, z=-0.007439), Rotation(pitch=0.059928, yaw=0.133172, roll=0.000000))


In [7]:
# Start/stop vehicle autopilot
vehicle.set_autopilot(True)
# vehicle.set_autopilot(False)

In [15]:
# Get map information
town_map = world.get_map()
print(town_map.name)

# Get waypoints
roads = town_map.get_topology()
print(roads)

Carla/Maps/Town10HD_Opt
[(<carla.libcarla.Waypoint object at 0x000001CD08F37D80>, <carla.libcarla.Waypoint object at 0x000001CD08F37F40>), (<carla.libcarla.Waypoint object at 0x000001CD07EB0040>, <carla.libcarla.Waypoint object at 0x000001CD07EB0120>), (<carla.libcarla.Waypoint object at 0x000001CD07EB0190>, <carla.libcarla.Waypoint object at 0x000001CD07EB0200>), (<carla.libcarla.Waypoint object at 0x000001CD07EB0270>, <carla.libcarla.Waypoint object at 0x000001CD07EB02E0>), (<carla.libcarla.Waypoint object at 0x000001CD07EB0350>, <carla.libcarla.Waypoint object at 0x000001CD07EB03C0>), (<carla.libcarla.Waypoint object at 0x000001CD07EB0430>, <carla.libcarla.Waypoint object at 0x000001CD07EB04A0>), (<carla.libcarla.Waypoint object at 0x000001CD07EB0580>, <carla.libcarla.Waypoint object at 0x000001CD07EB0510>), (<carla.libcarla.Waypoint object at 0x000001CD07EB05F0>, <carla.libcarla.Waypoint object at 0x000001CD07EB06D0>), (<carla.libcarla.Waypoint object at 0x000001CD07EB0740>, <carla

In [13]:
# Look at waypoint information. transform wrapped as a waypoint
print(roads[0][0])

Waypoint(Transform(Location(x=109.929878, y=-9.334196, z=0.000000), Rotation(pitch=0.000000, yaw=-89.609253, roll=0.000000)))


In [14]:
# Get amount of waypoints on the map
print(len(roads))

200


In [15]:
# Show waypoints on the map
for i in range(len(roads)):
    print(roads[i][0].transform.location)

Location(x=109.929878, y=-9.334196, z=0.000000)
Location(x=106.429962, y=-9.358066, z=0.000000)
Location(x=102.980186, y=-22.705795, z=0.000000)
Location(x=99.480568, y=-22.757402, z=0.000000)
Location(x=19.330637, y=140.959915, z=0.000000)
Location(x=19.350212, y=137.459961, z=0.000000)
Location(x=102.300583, y=82.914726, z=0.000000)
Location(x=98.800667, y=82.890854, z=0.000000)
Location(x=109.502762, y=53.293152, z=0.000000)
Location(x=106.002846, y=53.269283, z=0.000000)
Location(x=106.002846, y=53.269283, z=0.000000)
Location(x=106.002846, y=53.269283, z=0.000000)
Location(x=102.578484, y=42.165668, z=0.000000)
Location(x=99.078568, y=42.141800, z=0.000000)
Location(x=99.078568, y=42.141800, z=0.000000)
Location(x=109.896400, y=-4.425507, z=0.000000)
Location(x=106.396484, y=-4.449376, z=0.000000)
Location(x=102.930038, y=-9.381519, z=0.000000)
Location(x=99.430122, y=-9.405389, z=0.000000)
Location(x=99.430122, y=-9.405389, z=0.000000)
Location(x=-0.147736, y=130.210083, z=0.0000

In [13]:
# Make a route from one position to another
# pos 1 (start): Transform(Location(x=-64.644508, y=24.470905, z=-0.006843), Rotation(pitch=0.064163, yaw=0.159180, roll=-0.000671))
# pos 2 (end): Transform(Location(x=-45.162773, y=-40.666988, z=-0.007455), Rotation(pitch=0.064094, yaw=-89.594597, roll=0.000000))

#import code for route planning, need networkx and numpy modules
sys.path.append('C:\CARLA_0.9.15\PythonAPI\carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner

In [14]:
# plan route and visually draw it in the simulator
sampling_resolution = 2
grp = GlobalRoutePlanner(town_map, sampling_resolution)

# point_a = random.choice(roads)[0].transform.location
# point_b = random.choice(roads)[0].transform.location

# Use predefined points
point_a = carla.Location(x=-64.644508, y=24.470905, z=-0.006843)
point_b = carla.Location(x=-45.162773, y=-40.666988, z=-0.007455)

route = grp.trace_route(point_a, point_b)

# Draw route in simulator
for i in range(len(route)):
    world.debug.draw_string(route[i][0].transform.location, 'O', draw_shadow=False,
                            color=carla.Color(r=255, g=0, b=0), life_time=120.0,
                            persistent_lines=True)

NameError: name 'town_map' is not defined

In [1]:
# Clean up all vehicle actors and sensors using the cleanUp function
cleanUp()

NameError: name 'cleanUp' is not defined

In [17]:
# Create spectator over spawned vehicle
spectator = world.get_spectator()
spawn_points = world.get_map().get_spawn_points()
start_point = spawn_points[0]

spectator_pos = carla.Transform(start_point.location + carla.Location(x=20, y=10, z=4), 
                                carla.Rotation(yaw = start_point.rotation.yaw - 155))
spectator.set_transform(spectator_pos)